# 뉴스 기사 탐색 챗봇 Application

Colab 환경에서 뉴스 기사 탐색 챗봇 애플리케이션을 만들어봅시다. 애플리케이션 사용자의 유스케이스는 아래와 같습니다.


## 패키지 및 예제 데이터 다운로드하기
예제를 실행시키기 위해 python package들을 설치합니다. Colab에서 실행하지 않는 경우 이 셀은 실행하지 않습니다.

In [ ]:
!wget https://raw.githubusercontent.com/mentor1023/dl_apps/main/nlp/requirements-colab.txt
!pip install -r requirements-colab.txt

In [1]:
!wget https://raw.githubusercontent.com/mentor1023/dl_apps/main/nlp/requirements-colab.txt

--2024-12-04 08:10:48--  https://raw.githubusercontent.com/mentor1023/dl_apps/main/nlp/requirements-colab.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 114 [text/plain]
Saving to: ‘requirements-colab.txt’

requirements-colab. 100%[===================>]     114  --.-KB/s    in 0s      

2024-12-04 08:10:49 (1.66 MB/s) - ‘requirements-colab.txt’ saved [114/114]



In [2]:
!pip install -r requirements-colab.txt

INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.1/324.1 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5

## 패키지 불러오기

In [2]:
import json
from typing import Any, Callable, Dict, List, Tuple
import os

import gradio as gr
from openai import OpenAI
import requests
from bs4 import BeautifulSoup

## 챗봇 UI 구현하기

In [3]:
with gr.Blocks() as app:
    gr.Markdown("# 뉴스 기사 탐색 챗봇")
    gr.Markdown(
        """
        ## Chat
        얻고 싶은 정보에 대해 질문해보세요.
        """
    )
    chatbot = gr.Chatbot(label="Chat History")
    prompt = gr.Textbox(label="Input prompt")
    clear = gr.ClearButton([prompt, chatbot])

In [4]:
app.launch(inline=False, share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://e4383c327f9200d2d0.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [5]:
app.close()

Closing server running on port: 7860


## GPT API로 기본적인 챗봇 구현하기

In [6]:
os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"

In [7]:
os.environ["OPENAI_API_KEY"] = "본인의 OpenAI API Key 입력"

In [8]:
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
model = "gpt-3.5-turbo"

In [9]:
def respond(prompt: str, chat_history: List[str]) -> Tuple[str, List[str]]:
    messages = [{"role": "user", "content": prompt}]

    res = client.chat.completions.create(model=model, messages=messages)
    answer = res.choices[0].message.content

    chat_history.append((prompt, answer))

    return "", chat_history

In [10]:
with gr.Blocks() as app:
    gr.Markdown("# 뉴스 기사 탐색 챗봇")
    gr.Markdown(
        """
        ## Chat
        얻고 싶은 정보에 대해 질문해보세요.
        """
    )
    chatbot = gr.Chatbot(label="Chat History")
    prompt = gr.Textbox(label="Input prompt")
    clear = gr.ClearButton([prompt, chatbot])

    prompt.submit(respond, [prompt, chatbot], [prompt, chatbot])

In [11]:
app.launch(inline=False, share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://2b79d78e3389a0866e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [12]:
app.close()

Closing server running on port: 7860


## News API를 활용해 뉴스 기사 정보 가져오기

In [ ]:
os.environ["NEWS_API_KEY"] = "<NEWS_API_KEY>"

In [ ]:
endpoint = "https://newsapi.org/v2/everything"

In [ ]:
headers = {"x-api-key": os.environ["NEWS_API_KEY"]}
params = {
    "sources": "cnn",
    "language": "en",
    "q": "Tesla",
    "sortBy": "publishedAt"
}

# Fetch from newsapi.org
response = requests.get(endpoint, params=params, headers=headers)
data = response.json()

In [ ]:
data["articles"][0]

In [ ]:
class NewsApiClient:
    def __init__(self):
        self.news_api_key = os.environ["NEWS_API_KEY"]
        self.max_num_articles = 5

    def get_articles(
        self,
        query: str = None,
        from_date: str = None,
        to_date: str = None,
        sort_by: str = None,
    ) -> str:
        """Retrieve articles from newsapi.org (API key required)"""
        endpoint = "https://newsapi.org/v2/everything"
        headers = {"x-api-key": self.news_api_key}
        params = {
            "sortBy": "publishedAt",
            "sources": "cnn",
            "language": "en",
        }

        if query is not None:
            params["q"] = query
        if from_date is not None:
            params["from"] = from_date
        if to_date is not None:
            params["to"] = to_date
        if sort_by is not None:
            params["sortBy"] = sort_by

        # Fetch from newsapi.org
        # reference: https://newsapi.org/docs/endpoints/top-headlines
        response = requests.get(endpoint, params=params, headers=headers)
        data = response.json()

        if data["status"] == "ok":
            print(
                f"Processing {data['totalResults']} articles from newsapi.org. "
                + f"Max number is {self.max_num_articles}."
            )
            return json.dumps(
                data["articles"][: min(self.max_num_articles, len(data["articles"]))]
            )
        else:
            print("Request failed with message:", data["message"])
            return "No articles found"

In [ ]:
news_api_client = NewsApiClient()
news_api_client.get_articles(query="Tesla")

## 함수 호출 기능을 활용한 뉴스 기사 탐색 챗봇 구현하기

In [ ]:
signature_get_articles = {
    "name": "get_articles",
    "description": "Get news articles",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Freeform keywords or a phrase to search for.",
            },
            "from_date": {
                "type": "string",
                "description": "A date and optional time for the oldest article allowed. This should be in ISO 8601 format",
            },
            "to_date": {
                "type": "string",
                "description": "A date and optional time for the newest article allowed. This should be in ISO 8601 format",
            },
            "sort_by": {
                "type": "string",
                "description": "The order to sort the articles in",
                "enum": ["relevancy", "popularity", "publishedAt"],
            },
        },
        "required": [],
    },
}

In [ ]:
class GPTClient:
    def __init__(self):
        self.client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
        self.model = "gpt-3.5-turbo"

    def get_args_for_function_call(
        self, messages: List[Dict[str, str]], function_signatures: List[Dict[str, Any]]
    ) -> str:
        """
        If there is information for function in messages, get argument from messages.
        Otherwise get simple GPT response.
        """
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            functions=function_signatures,
        )
        return response.choices[0].message

    def request_with_function_call(
        self,
        messages: List[Dict[str, str]],
        function: Callable,
        function_call_resp,
        prompt: str = "",
    ) -> str:
        function_name = function_call_resp.function_call.name

        if prompt:
            messages.append({"role": "system", "content": prompt})

        # Run external function
        kwargs = json.loads(function_call_resp.function_call.arguments)
        function_result = function(**kwargs)

        # Append message
        messages.append(function_call_resp)
        messages.append(
            {"role": "function", "name": function_name, "content": function_result}
        )

        # GPT inference include function result
        res = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
        )
        return res.choices[0].message.content.strip()

In [ ]:
gpt_client = GPTClient()

In [ ]:
def respond(prompt: str, chat_history: List[str]) -> Tuple[str, List[str]]:
    # Get args from prompt
    messages = [{"role": "user", "content": prompt}]
    args_resp = gpt_client.get_args_for_function_call(messages, [signature_get_articles])

    # call functions requested by the model
    answer = args_resp.content
    title_list = []
    if args_resp.function_call:
        # GPT inference again with calling external function
        get_articles_prompt = """
            You are an assistant that provides news and headlines to user requests.
            Always try to get the articles using the available function calls.
            Write the arguments to your function at the top of your answer.
            Please output something like this:
            Number. [Title](Article Link)\n
                - Description: description\n
                - Publish Date: publish date\n
        """
        answer = gpt_client.request_with_function_call(
            messages=messages,
            function=news_api_client.get_articles,
            function_call_resp=args_resp,
            prompt=get_articles_prompt,
        )

    chat_history.append((prompt, answer))
    return "", chat_history

In [ ]:
with gr.Blocks() as app:
    gr.Markdown("# 뉴스 기사 탐색 챗봇")
    gr.Markdown(
        """
        ## Chat
        얻고 싶은 정보에 대해 질문해보세요.
        """
    )
    chatbot = gr.Chatbot(label="Chat History")
    prompt = gr.Textbox(label="Input prompt")
    clear = gr.ClearButton([prompt, chatbot])

    prompt.submit(respond, [prompt, chatbot], [prompt, chatbot])

In [ ]:
app.launch(inline=False, share=True)

In [ ]:
app.close()

## 뉴스 기사 요약 및 번역 UI 구현하기

In [ ]:
with gr.Blocks() as app:
    gr.Markdown("# 뉴스 기사 탐색 챗봇")
    with gr.Row():
        with gr.Column():
            gr.Markdown(
                """
                ## Chat
                얻고 싶은 정보에 대해 질문해보세요.
                """
            )
            chatbot = gr.Chatbot(label="Chat History")
            prompt = gr.Textbox(label="Input prompt")
            clear = gr.ClearButton([prompt, chatbot])

        with gr.Column():
            gr.Markdown(
                """
                ## Select News article
                원하는 기사를 선택하세요.
                """
            )
            article_list = gr.Dropdown(label="Article List", choices=None)
            abstract_box = gr.Textbox(
                label="Summarized article", lines=10, interactive=False
            )
            translate_box = gr.Textbox(
                label="Translated article", lines=10, interactive=False
            )
            crawl_btn = gr.Button("Get article!")

    prompt.submit(respond, [prompt, chatbot], [prompt, chatbot, article_list])

In [ ]:
app.launch(inline=False, share=True)

In [ ]:
app.close()

## 뉴스 제목 리스트를 선택 기능 구현하기

In [ ]:
signature_get_title_and_url = {
    "name": "get_title_and_url",
    "description": "Get title of article and url.",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {
                "type": "array",
                "description": "title array of articles",
                "items": {"type": "string", "description": "title of article"},
            },
            "url": {
                "type": "array",
                "description": "url array of articles",
                "items": {"type": "string", "description": "url of article"},
            },
        },
        "required": ["title", "url"],
    },
}

In [ ]:
TITLE_TO_URL = {}

def respond(prompt: str, chat_history: List[str]) -> Tuple[str, List[str]]:
    global TITLE_TO_URL

    # Get args from prompt
    messages = [{"role": "user", "content": prompt}]
    args_resp = gpt_client.get_args_for_function_call(messages, [signature_get_articles])

    # call functions requested by the model
    answer = args_resp.content
    title_list = []
    if args_resp.function_call:
        # GPT inference again with calling external function
        get_articles_prompt = """
            You are an assistant that provides news and headlines to user requests.
            Always try to get the articles using the available function calls.
            Write the arguments to your function at the top of your answer.
            Please output something like this:
            Number. [Title](Article Link)\n
                - Description: description\n
                - Publish Date: publish date\n
        """
        answer = gpt_client.request_with_function_call(
            messages=messages,
            function=news_api_client.get_articles,
            function_call_resp=args_resp,
            prompt=get_articles_prompt,
        )

        # Get titles and urls for dropdown from response message
        messages = [{"role": "user", "content": answer}]
        args_resp = gpt_client.get_args_for_function_call(
            messages, [signature_get_title_and_url]
        )
        args = json.loads(args_resp.function_call.arguments)
        title_list, url_list = args.get("title"), args.get("url")
        TITLE_TO_URL = {title: url for title, url in zip(title_list, url_list)}

    chat_history.append((prompt, answer))

    # Update dropdown
    drop_down = None
    if title_list:
        drop_down = gr.update(choices=title_list, interactive=True)

    return "", chat_history, drop_down

In [ ]:
with gr.Blocks() as app:
    gr.Markdown("# 뉴스 기사 탐색 챗봇")
    with gr.Row():
        with gr.Column():
            gr.Markdown(
                """
                ## Chat
                얻고 싶은 정보에 대해 질문해보세요.
                """
            )
            chatbot = gr.Chatbot(label="Chat History")
            prompt = gr.Textbox(label="Input prompt")
            clear = gr.ClearButton([prompt, chatbot])

        with gr.Column():
            gr.Markdown(
                """
                ## Select News article
                원하는 기사를 선택하세요.
                """
            )
            article_list = gr.Dropdown(label="Article List", choices=None)
            abstract_box = gr.Textbox(
                label="Summarized article", lines=10, interactive=False
            )
            translate_box = gr.Textbox(
                label="Translated article", lines=10, interactive=False
            )
            scrap_btn = gr.Button("Get article!")

    prompt.submit(respond, [prompt, chatbot], [prompt, chatbot, article_list])

In [ ]:
app.launch(inline=False, share=True)

In [ ]:
app.close()

## 뉴스 기사 스크래핑하기

In [ ]:
def scrap_cnn_article(url: str) -> Tuple[str, str]:
    """Scrap CNN news article."""
    rep = requests.get(url)

    soup = BeautifulSoup(rep.content, "html.parser")

    # Get main contents
    article = ""
    for paragraph in soup.find_all(["p", "h2"], {"class": ["paragraph", "subheader"]}):
        article += paragraph.text.strip()

    return article

In [ ]:
article = scrap_cnn_article("https://edition.cnn.com/2023/09/07/us/caste-discrimination-bill-california-passes-cec/index.html")

In [ ]:
article

## 뉴스 기사 요약 및 번역 기능 구현하기

In [ ]:
class GPTClient:
    def __init__(self):
        self.client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
        self.model = "gpt-3.5-turbo"

    def get_args_for_function_call(
        self, messages: List[Dict[str, str]], function_signatures: List[Dict[str, Any]]
    ) -> str:
        """
        If there is information for function in messages, get argument from messages.
        Otherwise get simple GPT response.
        """
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            functions=function_signatures,
        )
        return response.choices[0].message

    def request_with_function_call(
        self,
        messages: List[Dict[str, str]],
        function: Callable,
        function_call_resp,
        prompt: str = "",
    ) -> str:
        function_name = function_call_resp.function_call.name

        if prompt:
            messages.append({"role": "system", "content": prompt})

        # Run external function
        kwargs = json.loads(function_call_resp.function_call.arguments)
        function_result = function(**kwargs)

        # Append message
        messages.append(function_call_resp)
        messages.append(
            {"role": "function", "name": function_name, "content": function_result}
        )

        # GPT inference include function result
        res = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
        )
        return res.choices[0].message.content.strip()

    def summarize(self, texts: str) -> str:
        prompt = f"""
            Summarize the sentences '---' below.
            ---
            {texts}
            """
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ]

        # ChatGPT API 호출하기
        response = self.client.chat.completions.create(model=self.model, messages=messages)

        return response.choices[0].message.content

    def translate(self, texts: str) -> str:
        prompt = f"""
            Translate the sentences '---' below to Korean.
            ---
            {texts}
            """
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ]

        # ChatGPT API 호출하기
        response = self.client.chat.completions.create(model=self.model, messages=messages)

        return response.choices[0].message.content

gpt_client = GPTClient()

In [ ]:
def scrap_cnn_article(title: str) -> Tuple[str, str]:
    url = TITLE_TO_URL[title]
    rep = requests.get(url)

    soup = BeautifulSoup(rep.content, "html.parser")

    # Get main contents
    article = ""
    for paragraph in soup.find_all(["p", "h2"], {"class": ["paragraph", "subheader"]}):
        article += paragraph.text.strip()

    # Summarize and translate to Korean
    summarized_article = gpt_client.summarize(article)
    translated_article = gpt_client.translate(summarized_article)

    return summarized_article, translated_article

In [ ]:
with gr.Blocks() as app:
    gr.Markdown("# 뉴스 기사 탐색 챗봇")
    with gr.Row():
        with gr.Column():
            gr.Markdown(
                """
                ## Chat
                얻고 싶은 정보에 대해 질문해보세요.
                """
            )
            chatbot = gr.Chatbot(label="Chat History")
            prompt = gr.Textbox(label="Input prompt")
            clear = gr.ClearButton([prompt, chatbot])

        with gr.Column():
            gr.Markdown(
                """
                ## Select News article
                원하는 기사를 선택하세요.
                """
            )
            article_list = gr.Dropdown(label="Article List", choices=None)
            abstract_box = gr.Textbox(
                label="Summarized article", lines=10, interactive=False
            )
            translate_box = gr.Textbox(
                label="Translated article", lines=10, interactive=False
            )
            scrap_btn = gr.Button("Get article!")

    prompt.submit(respond, [prompt, chatbot], [prompt, chatbot, article_list])
    scrap_btn.click(
        scrap_cnn_article, inputs=[article_list], outputs=[abstract_box, translate_box]
    )

In [ ]:
app.launch(inline=False, share=True)

In [ ]:
app.close()

## 최종 App 구현

In [ ]:
import json
from typing import Any, Callable, Dict, List, Tuple

import gradio as gr
from openai import OpenAI
import requests
from bs4 import BeautifulSoup

In [ ]:
os.environ["OPENAI_API_KEY"] = "<OPENAI_API_KEY>"
os.environ["NEWS_API_KEY"] = "<NEWS_API_KEY>"

In [ ]:
class GPTClient:
    def __init__(self):
        self.client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
        self.model = "gpt-3.5-turbo"

    def get_args_for_function_call(
        self, messages: List[Dict[str, str]], function_signatures: List[Dict[str, Any]]
    ) -> str:
        """
        If there is information for function in messages, get argument from messages.
        Otherwise get simple GPT response.
        """
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            functions=function_signatures,
        )
        return response.choices[0].message

    def request_with_function_call(
        self,
        messages: List[Dict[str, str]],
        function: Callable,
        function_call_resp,
        prompt: str = "",
    ) -> str:
        function_name = function_call_resp.function_call.name

        if prompt:
            messages.append({"role": "system", "content": prompt})

        # Run external function
        kwargs = json.loads(function_call_resp.function_call.arguments)
        function_result = function(**kwargs)

        # Append message
        messages.append(function_call_resp)
        messages.append(
            {"role": "function", "name": function_name, "content": function_result}
        )

        # GPT inference include function result
        res = openai.ChatCompletion.create(
            model=self.model,
            messages=messages,
        )
        return res.choices[0].message.content.strip()

    def summarize(self, texts: str) -> str:
        prompt = f"""
            Summarize the sentences '---' below.
            ---
            {texts}
            """
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ]

        # ChatGPT API 호출하기
        response = self.client.chat.completions.create(model=self.model, messages=messages)

        return response.choices[0].message.content

    def translate(self, texts: str) -> str:
        prompt = f"""
            Translate the sentences '---' below to Korean.
            ---
            {texts}
            """
        messages = [
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt},
        ]

        # ChatGPT API 호출하기
        response = self.client.chat.completions.create(model=self.model, messages=messages)

        return response.choices[0].message.content

gpt_client = GPTClient()


class NewsApiClient:
    def __init__(self):
        self.news_api_key = os.environ["NEWS_API_KEY"]
        self.max_num_articles = 5

    def get_articles(
        self,
        query: str = None,
        from_date: str = None,
        to_date: str = None,
        sort_by: str = None,
    ) -> str:
        """Retrieve articles from newsapi.org (API key required)"""
        base_url = "https://newsapi.org/v2/everything"
        headers = {"x-api-key": self.news_api_key}
        params = {
            "sortBy": "publishedAt",
            "sources": "cnn",
            "language": "en",
        }

        if query is not None:
            params["q"] = query
        if from_date is not None:
            params["from"] = from_date
        if to_date is not None:
            params["to"] = to_date
        if sort_by is not None:
            params["sortBy"] = sort_by

        # Fetch from newsapi.org
        # reference: https://newsapi.org/docs/endpoints/top-headlines
        response = requests.get(base_url, params=params, headers=headers)
        data = response.json()

        if data["status"] == "ok":
            print(
                f"Processing {data['totalResults']} articles from newsapi.org. "
                + f"Max number is {self.max_num_articles}."
            )
            return json.dumps(
                data["articles"][: min(self.max_num_articles, len(data["articles"]))]
            )
        else:
            print("Request failed with message:", data["message"])
            return "No articles found"


news_api_client = NewsApiClient()
gpt_client = GPTClient()

In [ ]:
def scrap_cnn_article(title: str) -> Tuple[str, str]:
    url = TITLE_TO_URL[title]
    rep = requests.get(url)

    soup = BeautifulSoup(rep.content, "html.parser")

    # Get main contents
    article = ""
    for paragraph in soup.find_all(["p", "h2"], {"class": ["paragraph", "subheader"]}):
        article += paragraph.text.strip()

    # Summarize and translate to Korean
    summarized_article = gpt_client.summarize(article)
    translated_article = gpt_client.translate(summarized_article)

    return summarized_article, translated_article

In [ ]:
signature_get_articles = {
    "name": "get_articles",
    "description": "Get news articles",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "Freeform keywords or a phrase to search for.",
            },
            "from_date": {
                "type": "string",
                "description": "A date and optional time for the oldest article allowed. This should be in ISO 8601 format",
            },
            "to_date": {
                "type": "string",
                "description": "A date and optional time for the newest article allowed. This should be in ISO 8601 format",
            },
            "sort_by": {
                "type": "string",
                "description": "The order to sort the articles in",
                "enum": ["relevancy", "popularity", "publishedAt"],
            },
        },
        "required": [],
    },
}

signature_get_title_and_url = {
    "name": "get_title_and_url",
    "description": "Get title of article and url.",
    "parameters": {
        "type": "object",
        "properties": {
            "title": {
                "type": "array",
                "description": "title array of articles",
                "items": {"type": "string", "description": "title of article"},
            },
            "url": {
                "type": "array",
                "description": "url array of articles",
                "items": {"type": "string", "description": "url of article"},
            },
        },
        "required": ["title", "url"],
    },
}

In [ ]:
TITLE_TO_URL = {}

def respond(prompt: str, chat_history: List[str]) -> Tuple[str, List[str]]:
    global TITLE_TO_URL

    # Get args from prompt
    messages = [{"role": "user", "content": prompt}]
    args_resp = gpt_client.get_args_for_function_call(messages, [signature_get_articles])

    # call functions requested by the model
    answer = args_resp.content
    title_list = []
    if args_resp.function_call:
        # GPT inference again with calling external function
        get_articles_prompt = """
            You are an assistant that provides news and headlines to user requests.
            Always try to get the articles using the available function calls.
            Write the arguments to your function at the top of your answer.
            Please output something like this:
            Number. [Title](Article Link)\n
                - Description: description\n
                - Publish Date: publish date\n
        """
        answer = gpt_client.request_with_function_call(
            messages=messages,
            function=news_api_client.get_articles,
            function_call_resp=args_resp,
            prompt=get_articles_prompt,
        )

        # Get titles and urls for dropdown from response message
        messages = [{"role": "user", "content": answer}]
        args_resp = gpt_client.get_args_for_function_call(
            messages, [signature_get_title_and_url]
        )
        args = json.loads(args_resp.function_call.arguments)
        title_list, url_list = args.get("title"), args.get("url")
        TITLE_TO_URL = {title: url for title, url in zip(title_list, url_list)}

    chat_history.append((prompt, answer))

    # Update dropdown
    drop_down = None
    if title_list:
        drop_down = gr.update(choices=title_list, interactive=True)

    return "", chat_history, drop_down

In [ ]:
with gr.Blocks() as app:
    gr.Markdown("# 뉴스 기사 탐색 챗봇")
    with gr.Row():
        with gr.Column():
            gr.Markdown(
                """
                ## Chat
                얻고 싶은 정보에 대해 질문해보세요.
                """
            )
            chatbot = gr.Chatbot(label="Chat History")
            prompt = gr.Textbox(label="Input prompt")
            clear = gr.ClearButton([prompt, chatbot])

        with gr.Column():
            gr.Markdown(
                """
                ## Select News article
                원하는 기사를 선택하세요.
                """
            )
            article_list = gr.Dropdown(label="Article List", choices=None)
            abstract_box = gr.Textbox(
                label="Summarized article", lines=10, interactive=False
            )
            translate_box = gr.Textbox(
                label="Translated article", lines=10, interactive=False
            )
            scrap_btn = gr.Button("Get article!")

    prompt.submit(respond, [prompt, chatbot], [prompt, chatbot, article_list])
    scrap_btn.click(
        scrap_cnn_article, inputs=[article_list], outputs=[abstract_box, translate_box]
    )

In [ ]:
app.launch(inline=False, share=True)

In [ ]:
app.close()